In [1]:
import pandas as pd
from database import Database
from email_sender import EmailSender
from excel_writer import ExcelWriter
from sklearn.pipeline import Pipeline
from sqlalchemy import select
from ssrs import HotelData
from transformers import HotelDataEncoder, HotelDataReadCsv

In [2]:
db = Database()

hotel_mapping = pd.read_sql(
    sql=select(
        db.mapping_hotel.c.external_code,
        db.mapping_hotel.c.external_name,
        db.mapping_hotel.c.hotel_id,
    ),
    con=db.engine.connect(),
)

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
pipeline = Pipeline(
    [
        ("csv_reader", HotelDataReadCsv()),
        ("data_encoder", HotelDataEncoder()),
    ]
)

In [4]:
dfs = []

for destination in ["DU", "OM"]:
    data = HotelData(destination).get()
    df = pipeline.fit_transform(data)

    if destination == "DU":
        df = df[df["Country"].isin(["AE", "SA"])]

    dfs.append(df)

hotel_list = pd.concat(dfs, axis=0, ignore_index=True)

In [5]:
hotel_list = hotel_list.merge(
    hotel_mapping,
    how="left",
    left_on="HotelID",
    right_on="external_code",
)

In [6]:
filtered_rows = hotel_list[hotel_list["HotelName_GWG"] != hotel_list["external_name"]]

In [7]:
if not filtered_rows.empty:
    filtered_rows.name = "hotel_list"

    excel_writer = ExcelWriter([filtered_rows], "hotel_list.xlsx")
    excel_writer.write_to_excel()
    excel_writer.writer.close()

    sender = EmailSender(
        subject="Hotel Table Discrepancies: Action Required",
        to=[
            "ivan.orara@meetingpointuae.com",
            #"analyst-contracting@meetingpointuae.com",
        ],
    )

    sender.send_email(
        ["hotel_list.xlsx"],
        """
            <p>This is an automated report.</p>
            <p>Please do not reply.</p>
        """,
    )
else:
    print("No action required.")

No action required.


In [8]:
hotel_list.to_csv("hotels.csv", index=False)